# Almacenamiento de Agua

Importar librerías a usar.

- Numpy
- ee: API de Google Earth Engine
- folium: visualización de mapas sobre Leaflet

In [1]:
import numpy as np
import ee
import folium

Inicializar sesión de Google Earth Engine

In [2]:
ee.Initialize()

Parámetros de visualización (composición de bandas) para Sentinel-2

In [3]:
visRGB = {"bands": ["B4","B3","B2"],"min":0,"max":2000}
visRGB_water = {"bands": ["B4","B3","B2"],"min":0,"max":800}
visIndex = {"min":0,"max":1}

## Datos iniciales

### Fecha

- interestDate: Fecha deseada para la cual se desea calcular el volumen de agua almacenada.
- deltaDays: Rango de fechas para buscar imágenes adicionales +/- la fecha de interés.

In [4]:
interestDate = "2019-10-22"
deltaDays = 4

### Región

- xmin: longitud mínima.
- xmax: longitud máxima.
- ymin: latitud mínima.
- ymax: latitud máxima.

In [5]:
xmin = -8.22603391725042
ymin = 41.85962828770244
xmax = -8.063298931898858
ymax = 41.93092895284894

## Funciones

### 1. Visualización de una imagen en Folium

In [6]:
def foliumLayer(image,parameters = visRGB,layer_name = "layer"):
    
    folium_map = folium.Map(location = [centery,centerx],zoom_start = 13)
    
    mapIdDict = image.getMapId(parameters) # convertir imagen a id de visualizacion
    
    tile = folium.TileLayer(tiles = mapIdDict['tile_fetcher'].url_format,
                            attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
                            overlay = True,
                            name = layer_name)
    
    tile.add_to(folium_map)
    
    folium_map.add_child(folium.LayerControl())
    
    return folium_map

### 2. Cortar imágenes por ROI

In [7]:
def clip_images(image):
        
    return image.clip(ROI).copyProperties(image,["system:time_start"]) # retornar imagenes recortadas

### 3. Enmascarar nubes y sombras de una imagen

In [8]:
def clouds_shadows_mask(image):
    
    shadows_mask = image.select('SCL').eq(3).Not() # pixeles que no son sombra
    clouds_mask = image.select('SCL').lt(7).Or(image.select('SCL').gt(9)) # pixeles que no son nubes
    mask = shadows_mask.And(clouds_mask) # pixeles que no son ni sombra ni nubes
    
    return image.updateMask(mask).copyProperties(image,["system:time_start"]) # retornar imagenes enmascaradas

## Paso 1. Seleccionar imágenes

In [9]:
centerx = np.array([xmin,xmax]).mean()
centery = np.array([ymin,ymax]).mean()

ROI = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax]);

In [10]:
interestDate = np.datetime64(interestDate)
initialDate = np.datetime_as_string(interestDate - np.timedelta64(deltaDays,'D'))
finalDate = np.datetime_as_string(interestDate + np.timedelta64(deltaDays,'D'))

In [11]:
S2 = ee.ImageCollection("COPERNICUS/S2_SR").filterDate(initialDate,finalDate).filterBounds(ROI)

In [12]:
foliumLayer(S2.first(),layer_name = "Imagen sin mascara")

## Paso 2. Enmascarar nubes y sombras

In [13]:
S2_masked = S2.map(clouds_shadows_mask).map(clip_images)

In [14]:
foliumLayer(S2_masked.first(),layer_name = "Imagen con mascara")

## Paso 3. Reducir imagen

In [15]:
S2_reduced = S2_masked.median()

In [16]:
foliumLayer(S2_reduced,layer_name = "Imagen reducida")

## Paso 4. Calcular Índice de Vegetación

In [17]:
VI = S2_reduced.normalizedDifference(['B8','B3'])

In [18]:
foliumLayer(VI,visIndex,"Index")

## Paso 5. Segmentación

In [19]:
seeds = ee.Algorithms.Image.Segmentation.seedGrid(20)

SNIC = ee.Algorithms.Image.Segmentation.SNIC(image = ee.Image.cat([S2_reduced.select(['B2','B3','B4','B8']),VI]),
                                             size = 32,
                                             compactness = 1,
                                             connectivity = 8,
                                             neighborhoodSize = 256,
                                             seeds = seeds)

SNIC = SNIC.select(['B2_mean','B3_mean','B4_mean','B8_mean','nd_mean','clusters'], ['B2','B3','B4','B8','VI','clusters'])

In [20]:
foliumLayer(SNIC.select("clusters").randomVisualizer(),{},"Clusters")

## Paso 6. Clusterizar objetos por K-means

In [21]:
objectPropertiesImage = SNIC.select(['B2','B3','B4','B8','VI'])

X_train = objectPropertiesImage.sample(scale = 10,numPixels = 5000,region = ROI)

k = 3

kmeans = ee.Clusterer.wekaKMeans(k)
kmeans = kmeans.train(X_train)

clusterImage = objectPropertiesImage.cluster(kmeans)

In [22]:
foliumLayer(clusterImage.randomVisualizer(),{},"Kmeans")

## Paso 7. Seleccionar cluster de agua

In [23]:
values = []

for i in range(k):
    cluster_mask = clusterImage.eq(i)
    VI_clusterMasked = VI.updateMask(cluster_mask)
    mean_value = VI_clusterMasked.reduceRegion(reducer = ee.Reducer.mean(),geometry = ROI,scale = 50)
    values.append(mean_value.getInfo()['nd'])  

In [24]:
cluster_water = np.array(values).argmin().item()

In [25]:
water_mask = clusterImage.eq(cluster_water)
water = S2_reduced.updateMask(water_mask)

In [26]:
foliumLayer(water,visRGB_water,"Agua clasificada")

### Estimación de Batimetría

In [27]:
depth = water.select("B2").log().divide(water.select("B3").log())

In [28]:
foliumLayer(depth,{},"Depth")

In [29]:
kernel = ee.Kernel.square(radius = 2,normalize = False)

depth_median = depth.focal_mean(kernel = kernel,iterations = 1).updateMask(water_mask)

In [30]:
foliumLayer(depth_median,{},"Depth Median")